In [1]:
from IPython.display import display
import pandas as pd
import pickle

# stacked model

In [4]:
path_input_y = ''
path_input = ''
path_output = ''

In [ ]:
#layer1: single modality predictions
for fir_reg in ['svr','rf','xgb','eNet']:
    
    with open(path_input+fir_reg+'_layer1_output.pkl', 'rb') as f:
        dic = pickle.load(f)

    tb = {}
    for fold in ['Fold_0','Fold_1','Fold_2','Fold_3','Fold_4','Fold_5','Fold_6','Fold_7']:
        tb[fold] = pd.read_csv(path_input_y+fold+'/target_y_'+'test.csv', index_col=0, header=None)
        tb[fold].columns = ['y_true']
        tb[fold]['fold'] = fold[-1]
        
        for show_set in ['wm', 'lan', 'rel', 'mot', 'emo', 'soc', 'gam', 'rest-pca75', 'surf', 'VolBrain', 'subc', 'cort']:
            tb[fold][show_set] = dic[show_set]['test_pred'][fold]
            
    tb_all = tb['Fold_0']
    for fold in ['Fold_1','Fold_2','Fold_3','Fold_4','Fold_5','Fold_6','Fold_7']:
        tb_all = pd.merge(tb_all, tb[fold], how='outer')
    print('\n',fir_reg)
    display(tb_all)        
    
    tb_all.to_csv(path_output+'tables_'+fir_reg+'_test.csv')


In [ ]:
#layer2: stacked model predictions
for fir_reg in ['svr','rf','xgb','eNet']:
    for sec_reg in ['svr','rf','xgb','eNet']:
                
        tb = {}
        for fold in ['Fold_0','Fold_1','Fold_2','Fold_3','Fold_4','Fold_5','Fold_6','Fold_7']:
            tb[fold] = pd.read_csv(path_input_y+fold+'/target_y_'+'test.csv', index_col=0, header=None)
            tb[fold].columns = ['y_true']
            tb[fold]['fold'] = fold[-1]
                     
            for show_set in ['all','task','non-task','top-task']:
                with open(path_input+'layer2_'+show_set+'_'+fir_reg+'_'+sec_reg+'.pkl', 'rb') as f:
                    dic = pickle.load(f) 
                tb[fold][show_set] = dic['test_pred'][fold]
       
        tb_all = tb['Fold_0']
        for fold in ['Fold_1','Fold_2','Fold_3','Fold_4','Fold_5','Fold_6','Fold_7']:
            tb_all = pd.merge(tb_all, tb[fold], how='outer')
        display(tb_all)    
        
        tb_all.to_csv(path_output+'tables_'+fir_reg+'+'+sec_reg+'_test.csv')
        

# reliability test

In [ ]:
path_input_y = ''
path_output = ''

In [ ]:
timepoint = 1

#layer1:single modality
tb = pd.read_csv(path_input_y+'target_y_test'+str(timepoint)+'.csv', index_col=0, header=None)
tb.columns = ['y_true']

with open(path_input+'retest_layer1_output.pkl', 'rb') as f:
    data = pickle.load(f)
    
for fir_reg in ['svr','rf','xgb', 'eNet']:
        for show_set in ['wm', 'lan', 'rel', 'mot', 'emo', 'soc', 'gam', 'rest-pca75', 'surf', 'VolBrain', 'subc', 'cort']:
            tb[show_set] = data[fir_reg][show_set]['test'+str(timepoint)+'_pred']
        tb.to_csv(path_output+'tables_'+fir_reg+'_test'+str(timepoint)+'.csv')

#Layer2: stacked model 
tb = pd.read_csv(path_input_y+'target_y_test'+str(timepoint)+'.csv', index_col=0, header=None)
tb.columns = ['y_true']

with open(path_input+'retest_layer2_output.pkl', 'rb') as f:
    data = pickle.load(f)

for fir_reg in ['svr','rf','xgb', 'eNet']:
    for sec_reg in ['svr','rf','xgb', 'eNet']:
        for show_set in ['all', 'task', 'non-task', 'top-task']:
            tb[show_set] = data[fir_reg][sec_reg][show_set]['pred_test'+str(timepoint)]
        tb.to_csv(path_output+'tables_'+fir_reg+'+'+sec_reg+'_test'+str(timepoint)+'.csv')

# flat model

In [ ]:
path_input_y = ''
path_input = ''
path_output = ''

In [ ]:
for stacked_moda in ['all', 'task', 'top-task','non-task']:
    print('\n\n-------{}--------\n'.format(stacked_moda))
    for fir_reg in ['svr','rf','xgb','eNet']:
        print(fir_reg)
        with open(path_input+fir_reg+'-'+stacked_moda+'_flat_output.pkl', 'rb') as f:
            dic = pickle.load(f)

        tb = {}
        for fold in ['Fold_0','Fold_1','Fold_2','Fold_3','Fold_4','Fold_5','Fold_6','Fold_7']:
            tb[fold] = pd.read_csv(path_input_y+fold+'/target_y_test.csv', index_col=0, header=None)
            tb[fold].columns = ['y_true']
            tb[fold]['test_pred'] = dic['test_pred'][fold]

        tb_all = tb['Fold_0']
        for fold in ['Fold_1','Fold_2','Fold_3','Fold_4','Fold_5','Fold_6','Fold_7']:
            tb_all = pd.merge(tb_all, tb[fold], how='outer')

        display(tb_all)        
        tb_all.to_csv(path_output+'tables_'+fir_reg+'_'+stacked_moda+'.csv')